In [1]:
# Required Libraries
import numpy as np
import pandas as pd
import os
from stabilogram.stato import Stabilogram
from descriptors import compute_all_features

In [2]:
# Function to load the data
def load_data(foldername):
    base_path_posture = "P:/DATA_OCT_22/Expert_Eye/Dataset/gait_posture/raw_data/{}/t0/posture/"
    posture_data_files = {
        'yf_data': "2017-09-21_08_22_12_YF.txt", # Closed eyes
        'yo_data': "2017-09-21_08_22_12_YO.txt"  # Open eyes
    }
    try:
        data_posture = {}
        for name, filename in posture_data_files.items():
            path = base_path_posture.format(foldername) + filename
            data_posture[name] = pd.read_csv(path, sep="\t")

        return data_posture
    except:
        return None

In [3]:
# Get the list of folders in the raw_data directory
rootDir = 'p:/DATA_OCT_22/Expert_Eye/Dataset/gait_posture/raw_data'
folder_names = [name for name in os.listdir(rootDir) if os.path.isdir(os.path.join(rootDir, name))]

In [4]:
def compute_posture_features(folder_names, skip_folders=[]):
    # Dataframe to store all the features
    all_features_df = pd.DataFrame()

    # Loop over all the folder names
    for foldername in folder_names:
        # If the foldername is in the skip_folders list, skip this iteration
        if foldername in skip_folders:
            print(f"Skipping folder: {foldername}")
            continue
        try:
            print(f"Processing folder: {foldername}")

            rawdata = load_data(foldername)
            # If data is not loaded or if the data frames are empty,
            # add a row of NaNs to the DataFrame for this folder
            if rawdata is None or rawdata['yf_data'].empty or rawdata['yo_data'].empty:
                print(f"Folder '{foldername}' contains empty data files. Adding NaNs to DataFrame.")
                current_df = pd.DataFrame(index=[0])
                current_df['Foldername'] = foldername
                all_features_df = pd.concat([all_features_df, current_df], ignore_index=True)
                continue

            # Length and width of Nintendo wii balance board
            length = 53.6
            width = 33.7

            # The conditions identifiers
            eye_conditions = ['YF', 'YO']

            # A dictionary to store features for each eye condition
            features_dict = {}

            for i, condition in enumerate(eye_conditions):
                eye = rawdata[f'{condition.lower()}_data']

                # Calculate the total force and COP
                eye['TotalForce'] = (eye['BottomLeftCalcul_SensorsKG'] +
                    eye['BottomRightCalcul_SensorsKG'] +
                    eye['TopLeftCalcul_SensorsKG'] +
                    eye['TopRightCalcul_SensorsKG'])

                # Calculate COP_X and COP_Y
                eye['COP_X'] = ((eye['BottomLeftCalcul_SensorsKG'] +
                    eye['TopLeftCalcul_SensorsKG']) * width / 2 -
                    (eye['BottomRightCalcul_SensorsKG'] +
                    eye['TopRightCalcul_SensorsKG']) * width / 2) / eye['TotalForce']

                eye['COP_Y'] = ((eye['BottomLeftCalcul_SensorsKG'] +
                    eye['BottomRightCalcul_SensorsKG']) * length / 2 -
                    (eye['TopLeftCalcul_SensorsKG'] +
                    eye['TopRightCalcul_SensorsKG']) * length / 2) / eye['TotalForce']
                
                # Calculate the mean value of COP_X and COP_Y
                mean_COP_X = eye['COP_X'].mean()
                mean_COP_Y = eye['COP_Y'].mean()

                # Subtract the mean from each measurement to center the trajectories
                eye['COP_X_centered'] = eye['COP_X'] - mean_COP_X
                eye['COP_Y_centered'] = eye['COP_Y'] - mean_COP_Y

                time = eye['TIMESTAMP'].to_numpy()
                X = eye['COP_X_centered'].to_numpy()
                Y = eye['COP_Y_centered'].to_numpy()

                data = np.array([time, X, Y]).T

                # Verif if NaN data
                valid_index = (np.sum(np.isnan(data),axis=1) == 0)

                if np.sum(valid_index) != len(data):
                    raise ValueError("Clean NaN values first")

                stato = Stabilogram()
                stato.from_array(array=data)

                sway_density_radius = 0.3 # 3 mm

                params_dic = {"sway_density_radius": sway_density_radius}

                features = compute_all_features(stato, params_dic=params_dic)
                
                # Add the condition identifier to each key in the features dictionary
                features = {f'{k}_{condition}': v for k, v in features.items()}

                # Add the condition identifier to each key in the features dictionary
                features = {f'{k}_{condition}': v for k, v in features.items()}

                # Store the features for this condition in the features_dict
                features_dict[condition] = features

            # Combine features from all conditions into a single DataFrame
            all_features = pd.DataFrame({**features_dict['YF'], **features_dict['YO']}, index=[0])
            
            # Add the 'Foldername' to the all_features dataframe
            all_features['Foldername'] = foldername

            # Append the all_features dataframe to all_features_df
            all_features_df = pd.concat([all_features_df, all_features], ignore_index=True)

        except Exception as e:
            print(f"An error occurred while processing folder: {foldername}")
            print(f"Error: {e}")
            # optional: if you want to stop at the first error
            break

    # return the all_features_df
    return all_features_df


In [5]:
skip_folders = ['FRA005GMS', 'FRA008TJL', 'FRA010DMA', 'HUC001HMR', 'LEG007TGL', 'LEG018BPC', 'LEG027HJO', 'LEG034KLG', 'LEG042HJO', 'LEG047VSI', 'LEG048VHI', 'LEG049FAL', 'LEG050LMN']
posture_features = compute_posture_features(folder_names, skip_folders)


Processing folder: DUP002BME
Processing folder: FRA001AMA
Processing folder: FRA002CTI
Processing folder: FRA003BMG
Processing folder: FRA004LHN
Skipping folder: FRA005GMS
Processing folder: FRA006PJA
Processing folder: FRA007TTM
Skipping folder: FRA008TJL
Processing folder: FRA009RMI
Skipping folder: FRA010DMA
Processing folder: FRA011VJR
Processing folder: FRA012BJN
Processing folder: FRA013PMD
Processing folder: FRA014GJ
Processing folder: FRA015FMJ
Processing folder: FRA016GJ
Processing folder: FRA018ACV
Processing folder: FRA019AMA
Processing folder: FRA020THJ
Processing folder: FRA021LPA
Processing folder: FRA022RMT
Processing folder: FRA023CMC
Processing folder: FRA025HJM
Processing folder: FRA026RAL
Processing folder: FRA027MST
Processing folder: FRA028MJC
Processing folder: FRA029SCM
Processing folder: FRA030DHJ
Processing folder: FRA031AMJ
Processing folder: FRA032AEJ
Processing folder: FRA033SMC
Processing folder: FRA034GLM
Processing folder: FRA035GMD
Processing folder: FRA

In [1]:
posture_features.head()

NameError: name 'posture_features' is not defined

In [7]:
# Save the dataframe to an Excel file
#all_features_df.to_excel('all_features.xlsx')